# Исследование надежности заемщиков


## Изучение общей информации

In [1]:
import pandas as pd

try:
    data = pd.read_csv('/datasets/data.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')

In [2]:
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Предобработка данных

### Удаление пропусков

In [4]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [5]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### Обработка аномальных значений

In [6]:
data['days_employed'] = data['days_employed'].abs()

In [7]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, поэтому оставьте их как есть.

In [8]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

In [9]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

In [10]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5])

### Удаление пропусков (продолжение)

In [11]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

In [12]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Изменение типов данных

In [13]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

In [14]:
data['education'] = data['education'].str.lower()

In [15]:
data.duplicated().sum()

71

In [16]:
data = data.drop_duplicates()

### Категоризация данных

In [17]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [18]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

In [19]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

In [20]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [21]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

### Шаг 3. Исследование данных

#### 3.1 Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [22]:
# группируем данные по количеству детей и считаем количество элементов в каждой группе
group_of_children_total = data.groupby('children')['debt'].count()

In [23]:
# выводим сгруппированные данные
print(group_of_children_total)

children
0    14091
1     4808
2     2052
3      330
4       41
5        9
Name: debt, dtype: int64


**```Можно сразу увидеть, что размер групп неодинаковый (14 091 человек без детей против 9 человек с 5 детьми). Размер выборки без детей в 1566 раз больше, чем выборка людей с 5 детьми. Это говорит о том, что объемы выборки для исследования сильно отличаются друг от друга. Разброс выборки равен 14 082. Это достаточно большое значение, исследуемый признак(количество детей) сильно варьирует.
Слишком маленькая выборка(с 3, 4 и 5 детьми) не сможет обеспечить приемлемую точность результатов исследования, возрастает размер статистической ошибки. Ее не следует учитывать при анализе показателей.
К тому же, из генеральной выборки были удалены строки с аномальными значениями в столбце "children", это такие значения как -1 и 20, возможно, при ликвидировании аномалии, эти строки могли бы войти в одну из представленных выборок.```**

In [126]:
# считаем долю тех, кто имел задолженность по возврату кредитов, от общего количества людей в каждой группе
group_children_debt = (data.groupby('children')['debt'].sum() / group_of_children_total) * 100

In [127]:
# выводим данные
print(group_children_debt)

children
0    7.543822
1    9.234609
2    9.454191
3    8.181818
4    9.756098
5    0.000000
Name: debt, dtype: float64


Рассчитанные в последнем столбце значения(проценты) представляют собой удельные веса. Наибольший удельный вес должников по возврату кредита в общем объёме заемщиков в разрезе каждой группы имеют люди с двумя детьми и составляют 9,45% от общего числа заемщиков с двумя детьми. Наименьший удельный вес - у людей без детей, что составляет 7.54%.

In [26]:
# создадим сводную таблицу
# количество людей с задолженностью по возврату и без, сгруппированные по количеству детей
children_pivot = data.pivot_table(index=['children'], columns=['debt'], values='gender', aggfunc='count')

In [27]:
children_pivot

debt,0,1
children,,
0,13028.0,1063.0
1,4364.0,444.0
2,1858.0,194.0
3,303.0,27.0
4,37.0,4.0
5,9.0,NaN


**В ячейке у группы с 5 детьми, имеющими задолженность по выплате, пропущенное значение - NaN, опираясь на данные выше, мы видим, что в данной группе все 9 человек не имели долг. Обработаем пропущенное значение, заменив его на 0.**

In [119]:
# добавим в сводную таблицу столбец
# найдем долю людей с задолженностью среди тех, у кого задолженности не было, в разрезе каждой группы
children_pivot['share'] = (children_pivot[1] / children_pivot[0] ) * 100

# заменим пропуск в группе с 5 детьми
children_pivot[1][5] = 0

In [120]:
children_pivot

debt,0,1,share
children,,,
0,13028.0,1063.0,8.159349
1,4364.0,444.0,10.174152
2,1858.0,194.0,10.441335
3,303.0,27.0,8.910891
4,37.0,4.0,10.810811
5,9.0,0.0,0.000000


###### Зависимость между количеством детей и возвратом кредита в срок
**Вывод:** ```Зависимость между количеством детей и возвратом кредита в срок есть. 
Но в выборке очень большой разброс данных. Тяжело оценить корректность и точность полученных расчетов. Выборки с 3-5 детьми слишком малы, поэтому их статистические показатели не являются достоверными, они не будут учтены при анализе. 
Больше всего людей с задолженностью по возврату кредита в срок среди тех, кто имеет 2 детей. Это видно по относительному показателю координации (столбец share), он показывает соотношение людей с задолженностью и без между собой. В группе с двумя детьми на каждые 100 человек без задолженности по возврату приходится 10.4 человека с задолженностью.
Тогда как в группе без детей на каждые 100 человек без задолженности приходится всего 8 человек с задолженностью. 
Стоит отметить, что максимальное значение показателя - 10.44, больше минимального - 8.16 на 2.28 п.п., разрыв кажется небольшим на первый взгляд, но на деле разница между группами составляет 27.94%.  
Поэтому отсутствие детей положительно сказывается на платежеспособность заемщика.```

#### 3.2 Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [128]:
# группируем данные по семейному положению и считаем количество элементов в каждой группе по типу семейного положения
family_status_group = data.groupby('family_status')['debt'].count()

In [129]:
print(family_status_group.sort_values())

family_status
вдовец / вдова             951
в разводе                 1189
Не женат / не замужем     2796
гражданский брак          4134
женат / замужем          12261
Name: debt, dtype: int64


In [130]:
# считаем долю тех, кто имел задолженность по возврату кредитов, от общего количества людей в каждой группе 
family_status_debt = (data.groupby('family_status')['debt'].sum() / family_status_group) * 100

In [132]:
print(family_status_debt.sort_values())

family_status
вдовец / вдова           6.624606
в разводе                7.064760
женат / замужем          7.560558
гражданский брак         9.313014
Не женат / не замужем    9.763948
Name: debt, dtype: float64


Рассчитанные в последнем столбце значения(проценты) представляют собой удельные веса. Наибольшие удельные веса должников по возврату кредита в общем объёме заемщиков в разрезе каждой группы имеют люди незамужние или в гражданском браке и составляют 9,76% и 9.31% соответственно. Наименьший удельный вес - у вдовцов, что составляет 6.62%.

In [36]:
# создадим сводную таблицу
# количество людей с задолженностью по возврату и без, сгруппированные по семейному положению
family_status_pivot = data.pivot_table(index=['family_status'], columns=['debt'], values='gender', aggfunc='count')

In [37]:
family_status_pivot

debt,0,1
family_status,,
Не женат / не замужем,2523,273
в разводе,1105,84
вдовец / вдова,888,63
гражданский брак,3749,385
женат / замужем,11334,927


In [38]:
# добавим в сводную таблицу столбец
# найдем долю людей с задолженностью среди тех, у кого задолженности не было, в разрезе каждой группы
family_status_pivot['share'] = (family_status_pivot[1] / family_status_pivot[0] ) * 100

In [39]:
family_status_pivot

debt,0,1,share
family_status,,,
Не женат / не замужем,2523,273,10.820452
в разводе,1105,84,7.601810
вдовец / вдова,888,63,7.094595
гражданский брак,3749,385,10.269405
женат / замужем,11334,927,8.178931


###### Зависимость между семейным положением и возвратом кредита в срок
**Вывод:** ```Зависимость между семейным положением и возвратом кредита в срок есть. 
Больше всего людей с задолженностью по возврату кредита в срок среди неженатых/незамужних заемщиков, а также в гражданском браке. Это видно по относительному показателю координации (столбец share), он показывает соотношение людей с задолженностью и без между собой. В группе неженатых/незамужних на каждые 100 человек без задолженности по возврату приходится 10.8 человека с задолженностью.
В группе вдовцов/вдов на каждые 100 человек без задолженности приходится около 7 человек с задолженностью. Заемщики в разводе также имеют низкий показатель.
Стоит отметить, что максимальное значение показателя - 10.82, больше минимального - 7.09 на 3.73 п.п., разрыв кажется небольшим на первый взгляд, но на деле разница между группами составляет 52.61%.  
В связи с этим семейное положение также сказывается на платежеспособности заемщика.```

#### 3.3 Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [48]:
# выводим максимальный размер дохода
data['total_income'].max()

2265604

In [49]:
# выводим минимальный размер дохода
data['total_income'].min()

20667

Для определения зависимости выборку необходимо разделить на несколько интервалов. Интервалы должны быть равного размера, чтобы статистические показатели были наиболее корректными. Следует выделить несколько групп не на основе фактических числовых границ интервалов, а с использованием процентилей. Процентили поделят генеральную выборку на равные части, тем самым мы получим 4 равных интервала. 


In [74]:
# подключаем библиотеку и определяем процентили в столбце total_income
import numpy as np
np.percentile(data['total_income'], [25, 50, 75, 100])

array([ 107507.,  142594.,  195842., 2265604.])

In [75]:
# определяем интервалы для категоризации данных по уровню дохода
pd.qcut(data['total_income'], q=4)

0        (195842.0, 2265604.0]
1         (107507.0, 142594.0]
2         (142594.0, 195842.0]
3        (195842.0, 2265604.0]
4         (142594.0, 195842.0]
                 ...          
21520    (195842.0, 2265604.0]
21521     (142594.0, 195842.0]
21522    (20666.999, 107507.0]
21523    (195842.0, 2265604.0]
21524    (20666.999, 107507.0]
Name: total_income, Length: 21331, dtype: category
Categories (4, interval[float64]): [(20666.999, 107507.0] < (107507.0, 142594.0] < (142594.0, 195842.0] < (195842.0, 2265604.0]]

In [76]:
# проверим, действительно ли наши интервалы будут равны
data.loc[data['total_income'] < 107507.0]['total_income'].count()

5333

In [77]:
data.loc[(data['total_income'] < 142594) & (data['total_income'] >  107507.0)]['total_income'].count()

4384

In [78]:
data.loc[(data['total_income'] < 195842.0) & (data['total_income'] >  142594)]['total_income'].count()

5214

In [79]:
data.loc[data['total_income'] >  195842.0]['total_income'].count()

5332

Действительно, интервалы равны. Поделим наши данные на 4 категории по уровню дохода до 107507руб., до 142594руб., до 195842руб., выше 195842руб.

In [80]:
# создаем функцию для категоризации данных по уровню дохода
def division_by_income(income):
    try:
        if income <= 107507.0:
            return 'Низкий уровень дохода'
        elif income <= 142594.0:
            return 'Средний уровень дохода'
        elif income <= 195842.0:
            return 'Выше среднего уровень дохода'
        else:
            return 'Высокий уровень дохода'
    except:
        return 'Уровень дохода не вошел ни в одну из групп, где-то ошибка!'

In [86]:
# создаем столбец в датафрейме, с категоризацией данных 
data['total_income_group'] =  data['total_income'].apply(division_by_income)

In [88]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category,total_income_group
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью,Высокий уровень дохода
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем,Средний уровень дохода
2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью,Выше среднего уровень дохода
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования,Высокий уровень дохода
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы,Выше среднего уровень дохода


In [89]:
# считаем долю тех, кто имел задолженность по возврату кредитов, от общего количества людей в каждой категории дохода
total_income_debt = (data.groupby('total_income_group')['debt'].sum() / data.groupby('total_income_group')['debt'].count()) * 100

In [99]:
total_income_debt.sort_values()

total_income_group
Высокий уровень дохода          7.145536
Низкий уровень дохода           8.006750
Выше среднего уровень дохода    8.512270
Средний уровень дохода          8.807339
Name: debt, dtype: float64

In [100]:
# создадим сводную таблицу
# количество людей с задолженностью по возврату и без, сгруппированные по уровню дохода
total_income_pivot = data.pivot_table(index=['total_income_group'], columns=['debt'], values='gender', aggfunc='count')

In [101]:
total_income_pivot

debt,0,1
total_income_group,,
Высокий уровень дохода,4951,381
Выше среднего уровень дохода,4772,444
Низкий уровень дохода,4906,427
Средний уровень дохода,4970,480


In [102]:
# добавим в сводную таблицу столбец
# найдем долю людей с задолженностью среди тех, у кого задолженности не было, в разрезе каждой группы
total_income_pivot['share'] = (total_income_pivot[1] / total_income_pivot[0] ) * 100

In [103]:
total_income_pivot

debt,0,1,share
total_income_group,,,
Высокий уровень дохода,4951,381,7.695415
Выше среднего уровень дохода,4772,444,9.304275
Низкий уровень дохода,4906,427,8.703628
Средний уровень дохода,4970,480,9.657948


###### Зависимость между уровнем дохода и возвратом кредита в срок
**Вывод:** ```Зависимость между уровнем дохода  и возвратом кредита в срок есть. 
Больше всего людей с задолженностью по возврату кредита в срок у людей со средним уровнем дохода. Это видно по относительному показателю координации (столбец share), он показывает соотношение людей с задолженностью и без между собой. В группе заемщиков со средним уровнем дохода на каждые 100 человек без задолженности по возврату приходится 9.7 человека с задолженностью.
В группе с высоким уровнем дохода на каждые 100 человек без задолженности приходится около 7.7 человек с задолженностью, такие заемщики с меньшей вероятностью будут иметь просрочку по возврату долга. 
Стоит отметить, что максимальное значение показателя - 9.66, больше минимального - 7.70 на 1.96 п.п., разрыв кажется небольшим на первый взгляд, но на деле разница между группами составляет 25.62%.  
В связи с этим размер дохода имеет влияние на платежеспособность заемщика. ```

#### 3.4 Как разные цели кредита влияют на его возврат в срок?

In [104]:
# считаем долю тех, кто имел задолженность по возврату кредитов, от общего количества людей в каждой группе цели кредита
purpose_category_debt = (data.groupby('purpose_category')['debt'].sum() / data.groupby('purpose_category')['debt'].count()) * 100

In [105]:
purpose_category_debt.sort_values()

purpose_category
операции с недвижимостью    7.255139
проведение свадьбы          7.911803
получение образования       9.252758
операции с автомобилем      9.347978
Name: debt, dtype: float64

In [106]:
# создадим сводную таблицу
# количество людей с задолженностью по возврату и без, сгруппированные по уровню дохода
purpose_category_pivot = data.pivot_table(index=['purpose_category'], columns=['debt'], values='gender', aggfunc='count')

In [107]:
purpose_category_pivot

debt,0,1
purpose_category,,
операции с автомобилем,3879,400
операции с недвижимостью,9971,780
получение образования,3619,369
проведение свадьбы,2130,183


In [108]:
# добавим в сводную таблицу столбец
# найдем долю людей с задолженностью среди тех, у кого задолженности не было, в разрезе каждой группы
purpose_category_pivot['share'] = (purpose_category_pivot[1] / purpose_category_pivot[0] ) * 100

In [109]:
purpose_category_pivot

debt,0,1,share
purpose_category,,,
операции с автомобилем,3879,400,10.311936
операции с недвижимостью,9971,780,7.822686
получение образования,3619,369,10.196187
проведение свадьбы,2130,183,8.591549


###### Зависимость между целью кредита и возвратом кредита в срок
**Вывод:** ```Зависимость между целью кредита  и возвратом кредита в срок есть. 
Больше всего людей с задолженностью по возврату кредита в срок среди тех, кто берет кредит для операции с автомобилем и для получения образования. Это видно по относительному показателю координации (столбец share), он показывает соотношение людей с задолженностью и без между собой. В группах "операции с автомобилем" и "получение образования" на каждые 100 человек без задолженности по возврату приходится 10.3 и 10.2 человека с задолженностью соответственно.
В группе "операции с недвижимостью" на каждые 100 человек без задолженности приходится около 7.8 людей с задолженностью. Значит, риск задолженности по возврату кредита в этой категории граждан минимален.
Стоит отметить, что максимальное значение показателя - 10.31, больше минимального - 7.82 на 2.49 п.п., разрыв кажется небольшим на первый взгляд, но на деле разница между группами составляет 31.84%.  
В связи с этим цель кредитования оказывает влияние на платежеспособность заемщика.```

#### 3.5 Причины появления пропусков в исходных данных.

In [42]:
# выведем уникальные значения в столбце income_type
data['income_type'].unique()

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

*Ответ:* В исходных данных были пропуски в двух столбцах - days_employed и total_income. Количество пропусков в двух столбцах одинаково. Эти пропуски не являются полностью случайными. 
Они могут зависеть от значений столбца income_type(тип занятости). 
Например, если значение - пенсионер, при этом он не набрал нужный стаж для пенсии, то его доход будет равен 0, как и трудовой стаж. 
Также мы видим такие значения, как безработный и студент. Студент может работать неофициально, поэтому его трудовой стаж и доход равны 0, или не работать вовсе. Как в случае и с безработным.
Конечно, нельзя не учесть вероятность, что это может быть ошибка выгрузки, смены формата или записи данных.

#### 3.6 Почему заполнить пропуски медианным значением — лучшее решение для количественных переменных.

*Ответ:* Выбирая метод работы с пропусками, сначала стоит оценить разброс выборки, распределение наблюдений, ее симметричность. Если в данных много выдающихся значений, выборка ассиметрична, то наиболее точным будет медианное значение. Если выборка симметрична, нет выбросов, то лучше заполнить пропуски, используя среднее значение. Это связано с тем, что при ассиметричности выборки среднее значение будет сдвинуто и некорректно, так как эта метрика чувствительна к выбросам (тогда как медиана устойчива к ним).

### Шаг 4: общий вывод.

На возврат кредита в срок оказывают влияние количество детей, семейное положение, уровень дохода и цель кредита. Наибольшее влияние на платежеспособность оказывает семейное положение заещика.

Наилучшим заемщиком для банка будет вдовец/вдова или разведенный человек, без детей, с доходом от 195 842руб., с целью кредита - приобретение недвижимости. Такой заемщик имеет самую низкую вероятность нарушения срока возврата кредита.

Заемщиком с самым высоким риском по просрочке задолженности станет неженатый/незамужний человек, с двумя детьми и доходом в интервале от 107 507 до 142 594руб., с целью кредита - операции с автомобилем.